# Building communities

`micom` will construct communities from a specification via a Pandas DataFrame. Here, the DataFrame needs at least two columns: "id" and "file" which specify the ID of the organism/tissue and a file containing the actual individual model. 

To make more sense of that we can look at a small example. `micom` comes with a function that generates a simple example community specification consisting of several copies of a small *E. coli* model containing only the central carbon metabolism.

In [1]:
from micom.data import test_taxonomy

taxonomy = test_taxonomy()
taxonomy

,id,genus,species,reactions,metabolites,file
0,Escherichia_coli_1,Escherichia,Eschericia coli,95,72,/Users/cdiener/code/micom/micom/data/e_coli_co...
1,Escherichia_coli_2,Escherichia,Eschericia coli,95,72,/Users/cdiener/code/micom/micom/data/e_coli_co...
2,Escherichia_coli_3,Escherichia,Eschericia coli,95,72,/Users/cdiener/code/micom/micom/data/e_coli_co...
3,Escherichia_coli_4,Escherichia,Eschericia coli,95,72,/Users/cdiener/code/micom/micom/data/e_coli_co...
4,Escherichia_coli_5,Escherichia,Eschericia coli,95,72,/Users/cdiener/code/micom/micom/data/e_coli_co...


As we see this specification contains the required fields and some more information. In fact the specification may contain any number of additional information which will be saved along with the community model. One special example is "abundance" which we will get to know soon :) The logic behind requiring the community information in this table format is that this table can be appended as supplement to your project or publication as is and describes your community composition without any doubt.

In order to convert the specification in a community model we will use the `Community` class from `micom` which derives from the cobrapy `Model` class.

In [2]:
from micom import Community

com = Community(taxonomy)
print("Build a community with a total of {} reactions.".format(len(com.reactions)))

100%|██████████| 5/5 [00:02<00:00,  2.39models/s]

Build a community with a total of 495 reactions.


This includes the correctly scaled exchange reactions with the internal medium and initializes the external imports to the maximum found in all models. The original taxonomy is maintained in the `com.taxonomy` attribute.

Note that `micom` can figure out how to read a variety of different file types based on the extension. It curently supports:

- `.pickle` for pickled models
- `.xml` or `.gz` for XML models
- `.json` for JSON models
- `.mat` for COBRAtoolbox models


In [3]:
com.taxonomy

,id,genus,species,reactions,metabolites,file,abundance
id,,,,,,,
Escherichia_coli_1,Escherichia_coli_1,Escherichia,Eschericia coli,95,72,/Users/cdiener/code/micom/micom/data/e_coli_co...,0.2
Escherichia_coli_2,Escherichia_coli_2,Escherichia,Eschericia coli,95,72,/Users/cdiener/code/micom/micom/data/e_coli_co...,0.2
Escherichia_coli_3,Escherichia_coli_3,Escherichia,Eschericia coli,95,72,/Users/cdiener/code/micom/micom/data/e_coli_co...,0.2
Escherichia_coli_4,Escherichia_coli_4,Escherichia,Eschericia coli,95,72,/Users/cdiener/code/micom/micom/data/e_coli_co...,0.2
Escherichia_coli_5,Escherichia_coli_5,Escherichia,Eschericia coli,95,72,/Users/cdiener/code/micom/micom/data/e_coli_co...,0.2


As you can notice we have gained a new column called `abundance`. This column quantifies the relative quantity of each individual in the community. Since we did not specify this in the original taxonomy `micom` assumes that all individuals are present in the same quantity.

The presented community here is pretty simplistic. For microbial communities `micom` includes a larger taxonomy for 773 microbial species from the [AGORA paper](https://doi.org/10.1038/nbt.3703). Here the "file" column only contains the base names for the files but you can easily prepend any path as demonstrated in the following:

In [4]:
from micom.data import agora

tax = agora.copy()
tax.file = "models/" + tax.file  # assuming you have downloaded the AGORA models to the "models" folder
tax.head()

,organism,seedid,kingdom,phylum,mclass,order,family,genus,oxygenstat,metabolism,...,draftcreated,phenotype,imgid,ncbiid,platform,kbaseid,species,file,id,taxa_id
0,Abiotrophia defectiva ATCC 49176,Abiotrophia defectiva ATCC 49176 (592010.4),Bacteria,Firmicutes,Bacilli,Lactobacillales,Aerococcaceae,Abiotrophia,Facultative anaerobe,"Saccharolytic, fermentative or respiratory",...,07/01/14,1.0,2.562617e+09,592010.0,ModelSEED,NaN,defectiva,models/Abiotrophia_defectiva_ATCC_49176.xml,Abiotrophia_defectiva_ATCC_49176,46125.0
1,Achromobacter xylosoxidans A8,Achromobacter xylosoxidans A8 (762376.5),Bacteria,Proteobacteria,Betaproteobacteria,Burkholderiales,Alcaligenaceae,Achromobacter,Aerobe,Respiratory,...,NaN,NaN,NaN,762376.0,ModelSEED,kb_g_3268_model_gf,xylosoxidans,models/Achromobacter_xylosoxidans_A8.xml,Achromobacter_xylosoxidans_A8,85698.0
2,Achromobacter xylosoxidans NBRC 15126 = ATCC 2...,NaN,Bacteria,Proteobacteria,Betaproteobacteria,Burkholderiales,Alcaligenaceae,Achromobacter,Aerobe,Respiratory,...,NaN,NaN,NaN,1216976.0,Kbase,kb|g.208127,xylosoxidans,models/Achromobacter_xylosoxidans_NBRC_15126.xml,Achromobacter_xylosoxidans_NBRC_15126,85698.0
3,Acidaminococcus fermentans DSM 20731,Acidaminococcus fermentans DSM 20731 (591001.3),Bacteria,Firmicutes,Negativicutes,Acidaminococcales,Acidaminococcaceae,Acidaminococcus,Obligate anaerobe,Fermentative,...,04/17/16,NaN,6.463119e+08,591001.0,Kbase,kb|g.2555,fermentans,models/Acidaminococcus_fermentans_DSM_20731.xml,Acidaminococcus_fermentans_DSM_20731,905.0
4,Acidaminococcus intestini RyC-MR95,Acidaminococcus intestini RyC-MR95 (568816.4),Bacteria,Firmicutes,Negativicutes,Selenomonadales,Acidaminococcaceae,Acidaminococcus,Obligate anaerobe,"Asaccharolytic, glutamate is fermented",...,08/03/14,4.0,2.511231e+09,568816.0,Kbase,NaN,intestini,models/Acidaminococcus_intestini_RyC_MR95.xml,Acidaminococcus_intestini_RyC_MR95,187327.0


## Saving and loading communities

Contructing large community models can be slow which is due to performance limitations of the solvers. In essence, adding a single variable/constraint to a model qih 10 variables is much faster than adding it to a model with 10 million variables. Thus, we recommend you save the constructed community in a serialized format afterwards which will be much faster in loading repetitively.

In [5]:
%time com = Community(taxonomy)

100%|██████████| 5/5 [00:02<00:00,  2.39models/s]

CPU times: user 2.09 s, sys: 27.5 ms, total: 2.12 s
Wall time: 2.11 s


In [6]:
%time com.to_pickle("community.pickle")

CPU times: user 11.8 ms, sys: 5.29 ms, total: 17.1 ms
Wall time: 15.6 ms


In [7]:
from micom import load_pickle
%time com = load_pickle("community.pickle")

CPU times: user 101 ms, sys: 5.09 ms, total: 106 ms
Wall time: 104 ms


As we can see loading the model from the pickle format is much faster than creating it *de novo*.